# Lollipop Plot

A lollipop plot displays each element of a dataset as a segment and a circle. It is usually combined with the `count` stat, and is especially useful when you have several bars of the same height.

1. [Parameters `size`, `stroke` and `linewidth`](#stroke)

2. [Parameter `fatten`](#fatten)

3. [Horizontal Sticks](#direction)

4. [Sloped Baseline](#slope)

5. [Parameter `stat`](#stat)

6. [Lollipops in Marginal Layer](#ggmarginal)

7. [Lollipops and a Regression Line](#slope_and_intercept)


In [1]:
%useLatestDescriptors
%use dataframe
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.10.0. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.6.1.

In [3]:
import org.jetbrains.letsPlot.core.plot.base.stat.regression.LinearRegression
import kotlin.random.Random

In [4]:
val rnd = Random(0)
val data = mapOf(
    "x" to (-15..14).toList(),
    "y" to (0..29).map { rnd.nextDouble(1.0, 5.0) },
    "sugar" to (150..179).toList()
)

In [5]:
letsPlot(data) { x = "x"; y = "y" } + geomLollipop() + ggsize(600, 200)

-15 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

<a id="stroke"></a>

#### 1. Parameters `size`, `stroke` and `linewidth`

In [6]:
val plots = listOf(
    letsPlot(data) { x = "x"; y = "y"; size = "sugar" } + 
        geomLollipop() + ggtitle("variable 'size'"),
    letsPlot(data) { x = "x"; y = "y"; size = "sugar"; stroke = "sugar" } + 
        geomLollipop() + ggtitle("variable 'size' and 'stroke'"),
    letsPlot(data) { x = "x"; y = "y"; size = "sugar"; linewidth = "sugar" } + 
        geomLollipop() + ggtitle("variable 'size' and 'linewidth'")
)
    
gggrid(plots, ncol=1) + ggsize(800, 800)

-15 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 variable 'size' 
 
 
 
 
 y 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 sugar 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 155 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 165 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 170 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 175 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 variable 'size' and 'stroke' 
 
 
 
 
 y 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 sugar 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 155 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 165 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 170 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 175 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 variable 'size' and 'linewidth' 
 
 
 
 
 y 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 sugar 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 155 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 160 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 165 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 170 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 175

<a id="fatten"></a>

#### 2. Parameter `fatten`

In [7]:
gggrid(
    listOf(
        letsPlot(data) { x = "x"; y = "y"} + geomLollipop() + ggtitle("fatten=2.5 (default)"),
        letsPlot(data) { x = "x"; y = "y"} + geomLollipop(fatten = 5) + ggtitle("fatten=5"),
    )
)

-15 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 fatten=2.5 (default) 
 
 
 
 
 y 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 fatten=5 
 
 
 
 
 y 
 
 
 
 
 x

<a id="direction"></a>

#### 3. Horizontal Sticks

In [8]:
letsPlot(data) { x = "y"; y = "x" } + geomLollipop(dir = "h")

0 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 -15 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 x 
 
 
 
 
 y

<a id="slope"></a>

#### 4. Sloped Baseline

In [9]:
val slope = 0.5
val intercept = 1

val abline = letsPlot(data) { x = "x"; y = "y" } +
    geomABLine(intercept = intercept, slope = slope, color = "black", linetype = "dotted", size = 1.5) + 
    coordFixed(ylim = -12 to 12)
    
gggrid(
    listOf(
        abline + geomLollipop(intercept = intercept, slope = slope, shape = 21) + ggtitle("dir='v' (default)"),
        abline + geomLollipop(intercept = intercept, slope = slope, shape = 21, dir = "h") + ggtitle("dir='h'"),
        abline + geomLollipop(intercept = intercept, slope = slope, shape = 21, dir = "s") + ggtitle("dir='s'"),
    )
)

-10 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 dir='v' (default) 
 
 
 
 
 y 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 dir='h' 
 
 
 
 
 y 
 
 
 
 
 x 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 
 
 
 
 -5 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 dir='s' 
 
 
 
 
 y 
 
 
 
 
 x

<a id="stat"></a>

#### 5. Parameter `stat`

In [10]:
val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/mpg.csv")
df.head(3)

untitled,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.800000,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.800000,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.000000,2008,4,manual(m6),f,20,31,p,compact


In [11]:
gggrid(
    listOf(
        letsPlot(df.toMap()) { x = "class" } + geomLollipop(stat = Stat.count()) + ggtitle("stat='count'"),
        letsPlot(df.toMap()) { x = "hwy" } + geomLollipop(stat = Stat.bin()) + ggtitle("stat='bin'"),
        letsPlot(df.toMap()) { x = "hwy" } + geomLollipop(stat = Stat.density(n=30)) + ggtitle("stat='density'")
    )
)

compact 
 
 
 
 
 
 
 
 
 midsize 
 
 
 
 
 
 
 
 
 suv 
 
 
 
 
 
 
 
 
 2seater 
 
 
 
 
 
 
 
 
 minivan 
 
 
 
 
 
 
 
 
 pickup 
 
 
 
 
 
 
 
 
 subcompact 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 stat='count' 
 
 
 
 
 count 
 
 
 
 
 class 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 stat='bin' 
 
 
 
 
 count 
 
 
 
 
 hwy 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 0.01 
 
 
 
 
 
 
 0.02 
 
 
 
 
 
 
 0.03 
 
 
 
 
 
 
 0.04 
 
 
 
 
 
 
 0.05 
 
 
 
 
 
 
 0.06 
 
 
 
 
 
 
 0.07 
 
 
 
 
 
 
 
 
 stat='density' 
 
 
 
 
 density 
 
 
 
 
 hwy

<a id="ggmarginal"></a>

#### 6. Lollipops in Marginal Layer

In [12]:
letsPlot(df.toMap()) { x = "hwy"; y = "cty" } +
    geomBin2D(binWidth = 1 to 1) +
    ggmarginal("r", size=.2,
               layer = geomLollipop(stat = Stat.count(), orientation = "y", size = 1) { color = "..count.." } )

10 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 cty 
 
 
 
 
 hwy 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 
 
 
 
 count (color) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25

<a id="slope_and_intercept"></a>

#### 7. Lollipops and a Regression Line

In [13]:
val xs = df["hwy"].toList().map { (it as Number).toDouble() }
val ys = df["cty"].toList().map { (it as Number).toDouble() }

fun linearModel(x: Double) = LinearRegression.fit(xs, ys, 0.95)!!.evalX(x).y

val intercept = linearModel(0.0)
val slope = linearModel(1.0) - intercept

letsPlot(df.toMap()) { x = "hwy"; y = "cty" } +
    geomSmooth(level = 0.99, seed = 42) + 
    geomLollipop(slope = slope, intercept = intercept,
                 size = 1.2, shape = 21, color = "black", fill = "magenta") +
    coordFixed()

<path d="M18.774724389838383 314.6733689882237 L18.774724389838383 314.6733689882237 L23.527819172075937 311.57632876649086 L28.28091395431352 308.4780519940879 L33.03400873655107 305.37842727026606 L37.787103518788626 302.2773306026052 L42.54019830102621 299.17462379248394 L47.29329308326376 296.07015261408594 L52.046387865501345 292.96374476912695 L56.7994826477389 289.8552076029086 L61.55257742997645 286.7443255738359 L66.30567221221403 283.6308574795876 L71.05876699445156 280.5145334605865 L75.81186177668917 277.39505182767857 L80.56495655892672 274.2720757989109 L85.3180513411643 271.14523028325505 L90.07114612340186 268.0140989202222 L94.82424090563939 264.87822167570675 L99.577335687877 261.7370934056165 L104.33043047011452 258.59016392428873 L109.08352525235208 255.43684024019404 L113.83662003458969 252.27649172036055 L118.58971481682721 249.10845897531527 L123.34280959906476 245.93206716160716 L128.09590438130235 242.74664411671318 L132.84899916353996 239.55154322336287 L137.6020939457775 236.34617014597362 L142.35518872801507 233.13001167300487 L147.10828351025262 229.9026640209524 L151.86137829249017 226.66385737468613 L156.61447307472773 223.41347342168575 L161.36756785696534 220.1515533311369 L166.1206626392029 216.8782949572591 L170.87375742144044 213.59403969891554 L175.626852203678 210.29925098406 L180.3799469859156 206.99448737458627 L185.1330417681531 203.68037360898887 L189.88613655039072 200.35757256430415 L194.63923133262827 197.026760349213 L199.39232611486582 193.68860581692059 L204.14542089710338 190.34375494047592 L208.898515679341 186.99281985626504 L213.65161046157854 183.63637198869398 L218.40470524381604 180.2749384907677 L223.15780002605365 176.9090012126085 L227.9108948082912 173.53899748180822 L232.66398959052881 170.16532209633527 L237.41708437276637 166.78833005864016 L242.17017915500386 163.40833969887126 L246.92327393724148 160.02563593644658 L251.67636871947903 156.6404735100275 L256.42946350171667 153.25308006735253 L261.1825582839541 149.86365905123006 L265.93565306619166 146.47239234955254 L270.68874784842933 143.07944269859723 L275.4418426306669 139.6849558427761 L280.19493741290444 136.2890624625221 L284.948032195142 132.89187988677855 L289.70112697737954 129.49351360881076 L294.4542217596171 126.09405862464263 L299.20731654185465 122.69360061297954 L303.9604113240923 119.29221697440869 L308.7135061063299 115.88997774628126 L313.4666008885673 112.48694640815444 L318.219695670805 109.0831805911265 L322.9727904530424 105.67873270291705 L327.7258852352801 102.27365047915305 L332.47898001751764 98.86797747005949 L337.2320747997552 95.46175347060756 L341.98516958199286 92.05501490116222 L346.7382643642303 88.64779514476771 L351.49135914646786 85.24012484642498 L356.2444539287054 81.83203217902275 L360.99754871094297 78.42354307998266 L365.75064349318063 75.01468146215541 L370.5037382754182 71.60546940205103 L375.25683305765574 68.19592730809063 L380.0099278398933 64.78607407122524 L384.76302262213085 61.37592719997184 L389.5161174043684 57.96550294165684 L394.26921218660596 54.5548163914367 L394.26921218660596 72.52853183128047 L389.5161174043684 75.61265567049713 L384.76302262213085 78.69704180161892 L380.0099278398933 81.78170531980226 L375.25683305765574 84.86666247237372 L370.5037382754182 87.95193076785 L365.75064349318063 91.03752909718253 L360.99754871094297 94.12347786879207 L356.2444539287054 97.20979915918878 L351.49135914646786 100.29651688122334 L346.7382643642303 103.38365697231734 L341.98516958199286 106.47124760535957 L337.2320747997552 109.55931942535102 L332.47898001751764 112.647905815336 L327.7258852352801 115.73704319567923 L322.9727904530424 118.82677136135203 L318.219695670805 121.9171338625793 L313.4666008885673 125.00817843498817 L308.7135061063299 128.09995748629808 L303.9604113240923 131.1925286476075 L299.20731654185465 134.2859553984735 L294.4542217596171 137.3803077762472 L289.70112697737954 140.47566318151587 L284.948032195142 143.5721072929848 L280.19493741290